In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from pyrsm import gains, gains_plot, lift, lift_plot, confusion, profit_max, ROME_max

In [2]:
data = pd.read_csv('../intuit75k_new.csv')
categorical_columns=['zip_bins','sex','zip801','zip804']
othercol=['numords','last','dollars','sincepurch','bizflag','owntaxprod','version1','upgraded','training']
keep=categorical_columns+othercol+['label']
combind_data=data.loc[:,keep]
combind_data[categorical_columns] = combind_data[categorical_columns].apply(lambda x: LabelEncoder().fit_transform(x))
X_train=combind_data.loc[combind_data.training==1].drop(columns='label').drop(columns='training')
y_train=combind_data.loc[combind_data.training==1].label
X_test=combind_data.loc[combind_data.training==0].drop(columns='label').drop(columns='training')
y_test=combind_data.loc[combind_data.training==0].label

In [53]:
clf=RandomForestClassifier(n_estimators=150,random_state=123,max_depth=10)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [54]:
preds =clf.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

0.7651663235934608

In [55]:
margin=60
cost=1.41
breakeven_rate=cost/margin

testdata=combind_data.loc[combind_data.training==0]
testdata['randomforest']=preds[:,1]
testdata['pred_click']=1
testdata.loc[testdata['randomforest']<breakeven_rate,'pred_click']=0
tp=testdata.loc[testdata.pred_click==1].loc[testdata.label==1]
revenue=len(tp)*margin
pred_true=sum(testdata.pred_click)
totalcost=pred_true*cost
profit=revenue-totalcost
profit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

38013.75

In [56]:
#scaled profit
total=763334
testdata2=combind_data.loc[combind_data.training==0]
testdata2['randomforest']=preds[:,1]
testdata2['pred_click']=1
testdata2.loc[testdata2['randomforest']/2<breakeven_rate,'pred_click']=0
tp=testdata2.loc[testdata2.pred_click==1].loc[testdata2.label==1]
pred_true=sum(testdata2.pred_click)
pred_true_rate=pred_true/len(testdata2)
send_number=total*pred_true_rate
adj_response_rate=len(tp)/pred_true/2
exp_buyers=adj_response_rate*send_number
totalcost=send_number*cost
exp_profit=exp_buyers*margin-totalcost

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [57]:
exp_profit

447165.12831466686

#### Grid search for random forest

In [58]:
# Create the parameter grid: gbm_param_grid
rf_param_grid = {
    'n_estimators': range(100,301,50), ### how to set n_estimators?
    'max_depth': range(2,10)
}

# Instantiate the regressor: gbm
clf = RandomForestClassifier()

# Perform grid search: grid_auc
randomized_clf = RandomizedSearchCV(
    param_distributions=rf_param_grid,estimator=clf,scoring="roc_auc",n_iter=50,cv=5,verbose=1
)

# Fit grid_mse to the data
randomized_clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 13.2min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [60]:
print("Best parameters found: ", randomized_clf.best_params_)
print("higest auc found: ", np.abs(randomized_clf.best_score_))

Best parameters found:  {'n_estimators': 300, 'max_depth': 8}
higest auc found:  0.769589345811439


In [61]:
preds =randomized_clf.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

0.7643455706820561

In [62]:
#scaled profit

total=763334
testdata=225000
testdata2=combind_data.loc[combind_data.training==0]
testdata2['randomforest']=preds[:,1]
testdata2['pred_click']=1
testdata2.loc[testdata2['randomforest']/2<breakeven_rate,'pred_click']=0
tp=testdata2.loc[testdata2.pred_click==1].loc[testdata2.label==1]
pred_true=sum(testdata2.pred_click)
pred_true_rate=pred_true/len(testdata2)
send_number=total*pred_true_rate
adj_response_rate=len(tp)/pred_true/2
exp_buyers=adj_response_rate*send_number
totalcost=send_number*cost
exp_profit=exp_buyers*margin-totalcost

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [63]:
exp_profit

441591.772336

In [79]:
send_number=len(testdata2)*pred_true_rate
adj_response_rate=len(tp)/pred_true/2
exp_buyers=adj_response_rate*send_number
totalcost=send_number*cost
exp_profit_test=exp_buyers*margin-totalcost
exp_profit_test

13016.34